## 3.1 Implémentation des relations 1:1

### Exemple sous SQLAlchemy

In [1]:
from sqlalchemy import Column,  ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker,  relationship

# Moteur d'accès à la base sqlalchemy.sqlite
engine = create_engine('sqlite:///sqlalchemy.sqlite')

<img src="Relation_1_1_User_Office.png">

In [2]:
import random
import string

# Génération d'une chaîne de caractères aléatoire
def get_random_string(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

<p id="declarations" style="font-size:120%">Déclaration des classes métier :</p>

In [3]:
# Classe de base
Base = declarative_base()
Base.metadata.bind = engine

# Définition des classes
class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    pseudo = Column(String(16), nullable=False)
    email = Column(String(255), nullable=False)
    pwd = Column(String(32), nullable=False)
    
    office_id = Column(Integer,ForeignKey("office.id"))
    office = relationship("Office", uselist=False)

class Office(Base):
    __tablename__ = 'office'
    
    id = Column(Integer, primary_key=True)
    building = Column(String(32),nullable=False)
    floor = Column(Integer,nullable=False)
    room_id = Column(String(32),nullable=False)

<p id="usage" style="font-size:120%">Usage :</p>

In [4]:
# Session
DBSession = sessionmaker(bind=engine)
session = DBSession()

# Récupération d'un utilisateur
user = session.query(User).filter(User.pseudo == 'rde').one()

# Création d'un bureau
building='{}{}'.format(get_random_string(3),random.randint(10,99))
floor=random.randint(0,10)
room_id = get_random_string(8)
office = Office(building=building,floor=floor,room_id=room_id)

#Affectation du bureau à l'utilisateur
user.office = office

# Enregistrement de l'utilisateur et du bureau par la même occasion
session.add(user)
session.commit()

In [5]:
print(user.office)
print(office.id,office.building,office.floor,office.room_id)

5 qnq42 7 iskdnkqr


In [6]:
# Le bureau a bien été enregistré dans la base
office2 = session.query(Office).filter(Office.id == user.office.id).one()
print(office2.id,office2.building,office2.floor,office2.room_id)

5 qnq42 7 iskdnkqr


In [7]:
# On remet rde dans son bureau
office3 = session.query(Office).all()[0]
user.office = office3
session.add(user)
session.commit()